<a href="https://colab.research.google.com/github/MashaKubyshina/Analytics_and_reporting_scripts_for_work/blob/main/AB_Test_Cost_Per_Targeted_Subscriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **AB Test: Find cost per subscriber from FB ads in GoToVacc**

**Goal of this tempalte** is to facilitate reporting of GoToVacc (get out to vaccinate) metrics. This script allows to pull a customized report with key GoToVacc metrics from 2 different data scources in less than 5 minutes. 

This template can be used for any state. In this template I am using the state of Wisconsin.

**Key questions it will help you answer**

In this AB test we have 2 groups:
*   Group A: users geo targeted in FB ads (county targeting at ads level)
*   Group B: users without geo targeting in FB ads who got the zip code ask in the first step of the conversational flow.

We will be comparing cost per aquiring user A (user who entered the conversational flow from targeted FB ad) with the cost per acquiring user B (in this case we find user as a person who submtited their zip code).

This report will answer questions such as:


*   What is the number of zip codes submitted and cost per zip code in statewide and misc groups?
*   What is the cost per "geo-located" user in groups A and B?
*   What percent of users will take action?
*   What is the cost per action taker?
*   What is the number of vaccination response engagements?
*   How many users in our audience will share the vaccination message with their family and friends on FB?


**What you need before running the code**

To use this template you will need the following data sheets downloaded on your machine:


*   Chatfuel data in csv (select the user segment in people's tab using unique defining attribute and export users selecting all the attributes, click on "select all")
*   Facebook Ads data in xlsx (go to the ad campaign and create a report only using "ad set" as a parameter, don't include "ad name", pay attention to the date range you select in FB Ads, the results might be wrong or different if wrong dates are selected)


**Adding the data to your code**

After you have downloaded all the data to your machine, you will add it to your files folder by clicking on the file icon on the right. Click on the option "Upload to session storage" and upload both sheets here. The 2 data sheets will appear next to "sample data" folder.
Next you need to copy the path of each sheet by clicking on 3 dots next to each file name and clicking on "copy path". You will paste this path in the spaces provided in the top portion of this code.

Now you can go through the code and follow the instructions hashed in green.
 





**I deleted all the outputs in this script to protect PII and company data.**

In [23]:
# Import librarires

import pandas as pd
import numpy as np
from datetime import date

In [ ]:
# Copy the path from Chatfuel data by clicking on 3 dots next to the file name
# Paste this path in the place of the previous path, you will replace the "/content/2022_02_09_19_29_31.csv" bit)
# If needed please read the instructions above

cf_data=pd.read_csv("/content/2022_02_09_19_29_31.csv")
cf_data.head(5)

In [ ]:
# Copy the path from Facebook Ads data by clicking on 3 dots next to the file name
# Paste this path in the place of the previous path, you will replace the "/content/02_09_2022-Jan-1-2022-to-Feb-9-2022.xlsx" bit)
# If needed please read the instructions above

fb_data=pd.read_excel("/content/02_09_2022-Jan-1-2022-to-Feb-9-2022.xlsx")
fb_data.head(4)

**If you have succesfully copied and pasted the 3 paths, you can run the whole script.**
To run the script go to "Runtime" in the top menu and click on "Run all".
The script will pause when it will ask you to enter your authentification to mount the drive to place the produced report on it. You will find more instructions at that step.

In [ ]:
# Drop first row from the dataframe in Facebook ads data

fb_data.drop([0], inplace = True)
fb_data.head(3)

In [ ]:
# Grab the first row and make it the dataframe header in Facebook Ads data

new_header = fb_data.iloc[0] #grab the first row for the header
fb_data.columns = new_header #set the header row as the df header
fb_data.drop([1], inplace = True) # drop the first row (only use this line if it is an extra text header)
fb_data.head(10)

In [29]:
# Check columns in Facebook Ads data

fb_data.columns

Index([                        nan,               'Ad Set Name',
                               nan,           'Delivery status',
                  'Delivery level',             'Campaign name',
             'Attribution setting',               'Result type',
                         'Results',                     'Reach',
                     'Impressions',           'Cost per result',
                 'Quality ranking',   'Engagement rate ranking',
         'Conversion rate ranking',        'Amount spent (USD)',
       'New messaging connections',               'Link clicks',
                'Reporting starts',            'Reporting ends'],
      dtype='object', name=1)

In [30]:
# Rename certian columns in Facebook Ads data to avoid spaces

fb_data = fb_data.rename(columns={'Ad Name':'ad_name'})
fb_data = fb_data.rename(columns={'Campaign name':'campaign_name'})
fb_data = fb_data.rename(columns={'Ad Set Name':'adset_name'})
fb_data = fb_data.rename(columns={'New messaging connections':'new_messaging_connections'})
fb_data = fb_data.rename(columns={'Cost per result':'cost_per_result'})
fb_data = fb_data.rename(columns={'Amount spent (USD)':'amount_spent_usd'})
fb_data = fb_data.rename(columns={'Link Clicks':'link_clicks'})
fb_data.columns

Index([                        nan,                'adset_name',
                               nan,           'Delivery status',
                  'Delivery level',             'campaign_name',
             'Attribution setting',               'Result type',
                         'Results',                     'Reach',
                     'Impressions',           'cost_per_result',
                 'Quality ranking',   'Engagement rate ranking',
         'Conversion rate ranking',          'amount_spent_usd',
       'new_messaging_connections',               'Link clicks',
                'Reporting starts',            'Reporting ends'],
      dtype='object', name=1)

In [ ]:
# Delete rows in Facebook Ads Data where "results" and "adset_name" column values are NaN (this allows us to delete summary rows)

fb_data = fb_data.dropna(subset=['Results', 'adset_name'])
fb_data

In [ ]:
# Separate group A data (6 counties)

groupA_fb_data = fb_data[(fb_data['adset_name'] != 'WISC_Scaled_GoToVacc_Statewide') & (fb_data['adset_name'] != 'WISC_Scaled_GoToVacc_misc')] 
groupA_fb_data

In [ ]:
# Separate group B data (misc and statewide)

groupB_fb_data = fb_data[(fb_data['adset_name'] == 'WISC_Scaled_GoToVacc_Statewide') | (fb_data['adset_name'] == 'WISC_Scaled_GoToVacc_misc')] 
groupB_fb_data

In [ ]:
# Fb results sum of all adsets

fb_results=fb_data['Results'].sum()
fb_results

In [ ]:
# Check FB results for group A

groupA_fb_results=groupA_fb_data['Results'].sum()
groupA_fb_results

In [ ]:
# Check FB results for group B

groupB_fb_results=groupB_fb_data['Results'].sum()
int(groupB_fb_results)

In [ ]:
# Fb mean of all results

fb_cost_per_result_r=fb_data['cost_per_result'].mean()
fb_cost_per_result='${:0,.2f}'.format(fb_data['cost_per_result'].mean())
fb_cost_per_result

In [ ]:
# Fb mean of A group results

A_fb_cost_per_result_r=groupA_fb_data['cost_per_result'].mean()
A_fb_cost_per_result='${:0,.2f}'.format(groupA_fb_data['cost_per_result'].mean())
A_fb_cost_per_result

In [ ]:
# Fb mean of B group results

B_fb_cost_per_result_r=groupB_fb_data['cost_per_result'].mean()
B_fb_cost_per_result='${:0,.2f}'.format(groupB_fb_data['cost_per_result'].mean())
B_fb_cost_per_result

In [ ]:
# Alternative way to get cost per result (this should give the same number as we see on FB)

fb_cost_per_result_alt=fb_data['amount_spent_usd'].sum()/fb_data['Results'].sum()
fb_cost_per_result_alt
fb_cost_per_result_alt_for=fb_cost_per_result='${:0,.2f}'.format(fb_cost_per_result_alt)
fb_cost_per_result_alt_for

In [ ]:
# Alternative way to get cost per result group A

A_fb_cost_per_result_alt=groupA_fb_data['amount_spent_usd'].sum()/groupA_fb_data['Results'].sum()
A_fb_cost_per_result_alt
A_fb_cost_per_result_alt_for=groupA_fb_cost_per_result='${:0,.2f}'.format(A_fb_cost_per_result_alt)
A_fb_cost_per_result_alt_for

In [ ]:
# Alternative way to get cost per result group B

B_fb_cost_per_result_alt=groupB_fb_data['amount_spent_usd'].sum()/groupB_fb_data['Results'].sum()
B_fb_cost_per_result_alt
B_fb_cost_per_result_alt_for=groupB_fb_cost_per_result='${:0,.2f}'.format(B_fb_cost_per_result_alt)
B_fb_cost_per_result_alt_for

In [ ]:
# Fb ad spend all adsets

fb_total_ad_spend_r=fb_data['amount_spent_usd'].sum()
fb_total_ad_spend='${:0,.2f}'.format(fb_data['amount_spent_usd'].sum())
fb_total_ad_spend

In [ ]:
# Fb ad spend on groupA

A_fb_total_ad_spend_r=groupA_fb_data['amount_spent_usd'].sum()
A_fb_total_ad_spend='${:0,.2f}'.format(groupA_fb_data['amount_spent_usd'].sum())
A_fb_total_ad_spend

In [ ]:
# Fb ad spend on groupB

B_fb_total_ad_spend_r=groupB_fb_data['amount_spent_usd'].sum()
B_fb_total_ad_spend='${:0,.2f}'.format(groupB_fb_data['amount_spent_usd'].sum())
B_fb_total_ad_spend

In [ ]:
# Fb new messaging conversations started

fb_new_messaging_connections=fb_data['new_messaging_connections'].sum()
int(fb_new_messaging_connections)

In [ ]:
# Fb new messaging conversations started in A group

A_fb_new_messaging_connections=groupA_fb_data['new_messaging_connections'].sum()
A_fb_new_messaging_connections

In [ ]:
# Fb new messaging conversations started in B group

B_fb_new_messaging_connections=groupB_fb_data['new_messaging_connections'].sum()
B_fb_new_messaging_connections

In [ ]:
# Check all unique adset names

fb_data['adset_name'].unique()

In [ ]:
# Save total number of Chatfuel subscribers

total_cf_subscribers=len(cf_data)
total_cf_subscribers

In [ ]:
# Check the shape of the data

cf_data.shape

In [ ]:
# Check the county attribute in CF

cf_data['county_localyst'].value_counts()

In [ ]:
# Check the number of zip codes submitted

sum(cf_data['zip_code'].value_counts())

In [ ]:
# Aeparate A data in CF

A_cf_data = cf_data[(cf_data['adset_name'] != 'WISC_Scaled_GoToVacc_Statewide') & (cf_data['adset_name'] != 'WISC_Scaled_GoToVacc_misc')]
A_cf_data

In [ ]:
# Separate B data in CF

B_cf_data = cf_data[(cf_data['adset_name'] == 'WISC_Scaled_GoToVacc_Statewide') | (cf_data['adset_name'] == 'WISC_Scaled_GoToVacc_misc')]
B_cf_data

In [ ]:
# Save total number of Chatfuel subscribers in group A

A_total_cf_subscribers=len(A_cf_data)
A_total_cf_subscribers

In [ ]:
# Save total number of Chatfuel subscribers in group B

B_total_cf_subscribers=len(B_cf_data)
B_total_cf_subscribers

In [ ]:
# Check the values for vaccination status attribute in CF

cf_data['Vaccinated_Wisc_from_ad'].value_counts()

In [ ]:
# Check the values for vaccination status attribute in CF in group A

A_cf_data['Vaccinated_Wisc_from_ad'].value_counts()

In [ ]:
# Check the values for vaccination status attribute in CF in group B

B_cf_data['Vaccinated_Wisc_from_ad'].value_counts()

In [ ]:
# Check the county attribute in CF

cf_data['county_localyst'].value_counts()

In [ ]:
# Check the county attribute in CF in group A

A_cf_data['county_localyst'].value_counts()

In [ ]:
cf_data['wi_GTVac_action'].value_counts()

In [ ]:
cf_data['share_ask_received'].value_counts()

In [ ]:
# Save total number of live chat users

live_chat=sum(cf_data['live_chat_test'].value_counts())
live_chat

In [ ]:
# Save total number of zip codes submitted

zip_code=sum(cf_data['zip_code'].value_counts())
zip_code

In [ ]:
# Save total number of share agrees
share_agree=sum(cf_data['share_card_received'].value_counts())
share_agree

In [ ]:
# Save total number of share agrees in group A

A_share_agree=sum(A_cf_data['share_card_received'].value_counts())
A_share_agree

In [ ]:
# Save total number of share agrees in group B

B_share_agree=sum(B_cf_data['share_card_received'].value_counts())
B_share_agree

In [ ]:
# Clean the zip codes column (fill NA, make it type integer)

cf_data.zip_code=cf_data.zip_code.fillna(0.0).astype(int)
cf_data.zip_code

In [ ]:
# Save a slice of data with zip codes that are digits (not words)

zip_code_list=cf_data[(cf_data['zip_code'].apply(lambda x: str(x).isdigit())) & (cf_data['zip_code'] != 0)] 
zip_code_list

In [ ]:
# unique zip codes

len(zip_code_list['zip_code'].unique())

In [ ]:
# Save total number of phone numbers submitted as digits

zip_codes_submitted=len(zip_code_list['zip_code'])
zip_codes_submitted

In [ ]:
A_cf_data['wi_GTVac_action'].value_counts()

In [ ]:
B_cf_data['wi_GTVac_action'].value_counts()

In [ ]:
cf_data['wi_GTVac_action'].value_counts()

In [ ]:
vaccinated_users=len(cf_data[cf_data['wi_GTVac_action']== 'already_vaccinated']) + len(cf_data[cf_data['wi_GTVac_action']== 'already_vaccinated_$100_not_eligible'])+len(cf_data[cf_data['wi_GTVac_action']== 'already_vaccinated_$100'])
vaccinated_users

In [ ]:
A_vaccinated_users=len(A_cf_data[A_cf_data['wi_GTVac_action']== 'already_vaccinated']) + len(A_cf_data[A_cf_data['wi_GTVac_action']== 'already_vaccinated_$100_not_eligible'])+len(A_cf_data[A_cf_data['wi_GTVac_action']== 'already_vaccinated_$100'])
A_vaccinated_users

In [ ]:
B_vaccinated_users=len(B_cf_data[B_cf_data['wi_GTVac_action']== 'already_vaccinated']) + len(B_cf_data[B_cf_data['wi_GTVac_action']== 'already_vaccinated_$100_not_eligible'])+len(B_cf_data[B_cf_data['wi_GTVac_action']== 'already_vaccinated_$100'])
B_vaccinated_users

In [ ]:
# Filter for vaccianted that clicked on "I will share"

filtered_values = cf_data[(cf_data['wi_GTVac_action']=='already_vaccinated') & (cf_data['share_card_received'] != 'not set')]
filtered_values['share_card_received'].value_counts()

In [ ]:
# Filter for vaccianted that clicked on "I will share"

A_filtered_values = A_cf_data[(A_cf_data['wi_GTVac_action']=='already_vaccinated') & (A_cf_data['share_card_received'] != 'not set')]
A_filtered_values['share_card_received'].value_counts()

In [ ]:
# Filter for vaccianted that clicked on "I will share"

B_filtered_values = B_cf_data[(B_cf_data['wi_GTVac_action']=='already_vaccinated') & (B_cf_data['share_card_received'] != 'not set')]
B_filtered_values['share_card_received'].value_counts()

In [ ]:
# Total vaccinated action takers

vaccinated_action_takers=sum(filtered_values['share_card_received'].value_counts())
vaccinated_action_takers

In [ ]:
# Total vaccinated action takers in group A

A_vaccinated_action_takers=sum(A_filtered_values['share_card_received'].value_counts())
A_vaccinated_action_takers

In [ ]:
# Total vaccinated action takers in group B

B_vaccinated_action_takers=sum(B_filtered_values['share_card_received'].value_counts())
B_vaccinated_action_takers

In [ ]:
# Total of not vaccinated action takers

not_vac_action_takers=len(cf_data[cf_data['wi_GTVac_action']== 'need_vaccine_info']) + len(cf_data[cf_data['wi_GTVac_action']== 'need_vaccine_appt'])+len(cf_data[cf_data['wi_GTVac_action']== 'have_appointment'])
not_vac_action_takers

In [ ]:
# Total of not vaccinated action takers in group A

A_not_vac_action_takers=len(A_cf_data[A_cf_data['wi_GTVac_action']== 'need_vaccine_info']) + len(A_cf_data[A_cf_data['wi_GTVac_action']== 'need_vaccine_appt'])+len(A_cf_data[A_cf_data['wi_GTVac_action']== 'have_appointment'])
A_not_vac_action_takers

In [ ]:
# Total of not vaccinated action takers in group B

B_not_vac_action_takers=len(B_cf_data[B_cf_data['wi_GTVac_action']== 'need_vaccine_info']) + len(B_cf_data[B_cf_data['wi_GTVac_action']== 'need_vaccine_appt'])+len(B_cf_data[B_cf_data['wi_GTVac_action']== 'have_appointment'])
B_not_vac_action_takers

In [ ]:
total_action_takers=not_vac_action_takers+vaccinated_action_takers
total_action_takers

In [ ]:
A_total_action_takers=A_not_vac_action_takers+A_vaccinated_action_takers
A_total_action_takers

In [ ]:
B_total_action_takers=B_not_vac_action_takers+B_vaccinated_action_takers
B_total_action_takers

In [93]:
# Create, format and save all the values we need for our report

vaccinated_cf=len(cf_data[cf_data['Vaccinated_Wisc_from_ad']== 'yes'])
not_vaccinated_cf=len(cf_data[cf_data['Vaccinated_Wisc_from_ad']== 'no'])
total_subscribers=vaccinated_cf+not_vaccinated_cf
percent_vaccinated='{:.0%}'.format(vaccinated_cf/total_subscribers)
percent_not_vaccinated='{:.0%}'.format(not_vaccinated_cf/total_subscribers)
cost_per_action_taker='${:0,.2f}'.format(fb_total_ad_spend_r/total_action_takers)
percent_per_action_taker='{:.0%}'.format(total_action_takers/total_cf_subscribers)
percent_per_action_taker_vac='{:.0%}'.format(vaccinated_action_takers/vaccinated_cf)
percent_per_action_taker_not_vac='{:.0%}'.format(not_vac_action_takers/not_vaccinated_cf)
cost_per_result_cf='${:0,.2f}'.format(fb_total_ad_spend_r/total_cf_subscribers)
cost_per_share_agree='${:0,.2f}'.format(fb_total_ad_spend_r/share_agree)
percent_share_agree='{:.0%}'.format(share_agree/total_cf_subscribers)

In [94]:
# Create, format and save all the values we need for our report for A subset

A_vaccinated_cf=len(A_cf_data[A_cf_data['Vaccinated_Wisc_from_ad']== 'yes'])
A_not_vaccinated_cf=len(A_cf_data[A_cf_data['Vaccinated_Wisc_from_ad']== 'no'])
A_total_subscribers=A_vaccinated_cf+A_not_vaccinated_cf
A_percent_vaccinated='{:.0%}'.format(A_vaccinated_cf/A_total_subscribers)
A_percent_not_vaccinated='{:.0%}'.format(A_not_vaccinated_cf/A_total_subscribers)
A_cost_per_action_taker='${:0,.2f}'.format(A_fb_total_ad_spend_r/A_total_action_takers)
A_percent_per_action_taker='{:.0%}'.format(A_total_action_takers/A_total_cf_subscribers)
A_percent_per_action_taker_vac='{:.0%}'.format(A_vaccinated_action_takers/A_vaccinated_cf)
A_percent_per_action_taker_not_vac='{:.0%}'.format(A_not_vac_action_takers/A_not_vaccinated_cf)
A_cost_per_result_cf='${:0,.2f}'.format(A_fb_total_ad_spend_r/A_total_cf_subscribers)
A_cost_per_share_agree='${:0,.2f}'.format(A_fb_total_ad_spend_r/A_share_agree)
A_percent_share_agree='{:.0%}'.format(A_share_agree/A_total_cf_subscribers)

In [95]:
# Create, format and save all the values we need for our report for B subset

B_vaccinated_cf=len(B_cf_data[B_cf_data['Vaccinated_Wisc_from_ad']== 'yes'])
B_not_vaccinated_cf=len(B_cf_data[B_cf_data['Vaccinated_Wisc_from_ad']== 'no'])
B_total_subscribers=B_vaccinated_cf+B_not_vaccinated_cf
B_percent_vaccinated='{:.0%}'.format(B_vaccinated_cf/B_total_subscribers)
B_percent_not_vaccinated='{:.0%}'.format(B_not_vaccinated_cf/B_total_subscribers)
B_cost_per_action_taker='${:0,.2f}'.format(B_fb_total_ad_spend_r/B_total_action_takers)
B_percent_per_action_taker='{:.0%}'.format(B_total_action_takers/B_total_cf_subscribers)
B_percent_per_action_taker_vac='{:.0%}'.format(B_vaccinated_action_takers/B_vaccinated_cf)
B_percent_per_action_taker_not_vac='{:.0%}'.format(B_not_vac_action_takers/B_not_vaccinated_cf)
B_cost_per_result_cf='${:0,.2f}'.format(B_fb_total_ad_spend_r/B_total_cf_subscribers)
B_cost_per_share_agree='${:0,.2f}'.format(B_fb_total_ad_spend_r/B_share_agree)
B_percent_share_agree='{:.0%}'.format(B_share_agree/B_total_cf_subscribers)
B_cost_per_zip_code='${:0,.2f}'.format(B_fb_total_ad_spend_r/zip_codes_submitted)
B_percent_zip_codes_submitted='{:.0%}'.format(zip_codes_submitted/B_total_cf_subscribers)

In [ ]:
# initialize list of lists for the report

new_df = [
          ['Total Ad Spend (USD)', fb_total_ad_spend],
          ['Total Subscribers Acquired FB (Results)', int(fb_results)],
          ['Total Subscribers Acquired CF', total_cf_subscribers],
          ['Cost Per Result FB Ads (USD)', fb_cost_per_result_alt_for],
          ['Cost Per Acquisition CF', cost_per_result_cf],
          ['Total Vaccinated', vaccinated_cf],
          ['% of Vaccinated', percent_vaccinated],
          ['Total Not Vaccinated', not_vaccinated_cf],
          ['% of Not Vaccinated', percent_not_vaccinated],
          ['Vaccinated Action Takers', vaccinated_action_takers],
          ['% of Vaccinated Action Takerse (from Vaccinated Users)', percent_per_action_taker_vac],
          ['Not Vaccinated Action Takers', not_vac_action_takers],
          ['% of Not Vaccinated Action Takers (from Not Vaccinated Users', percent_per_action_taker_not_vac],
          ['Total Action Takers', total_action_takers],
          ['% of Action Takers', percent_per_action_taker],
          ['Cost Per Action Taker', cost_per_action_taker],
          ['Total Agree to Relational Shares', share_agree],
          ['% of Total Agree to Relational Shares', percent_share_agree],
          ['Cost Per Agree to Relational share', cost_per_share_agree],
          ['Live Chat engagements', live_chat],
          
          
          
        ]
 
# Create the pandas DataFrame
GoToVacc_metrics = pd.DataFrame(new_df, columns = ['Description', 'Metric'])
GoToVacc_metrics

In [ ]:
# Grou A report generation

A_new_df = [
          ['Total Ad Spend (USD)', A_fb_total_ad_spend],
          ['Total Subscribers Acquired FB (Results)', int(groupA_fb_results)],
          ['Total Subscribers Acquired CF', A_total_cf_subscribers],
          ['Cost Per Result FB Ads (USD)', A_fb_cost_per_result_alt_for],
          ['Cost Per Acquisition CF', A_cost_per_result_cf],

          ['Total Vaccinated', A_vaccinated_cf],
          ['% of Vaccinated', A_percent_vaccinated],
          ['Total Not Vaccinated', A_not_vaccinated_cf],
          ['% of Not Vaccinated', A_percent_not_vaccinated],
          ['Vaccinated Action Takers', A_vaccinated_action_takers],
          ['% of Vaccinated Action Takerse (from Vaccinated Users)', A_percent_per_action_taker_vac],
          ['Not Vaccinated Action Takers', A_not_vac_action_takers],
          ['% of Not Vaccinated Action Takers (from Not Vaccinated Users', A_percent_per_action_taker_not_vac],
          ['Total Action Takers', A_total_action_takers],
          ['% of Action Takers', A_percent_per_action_taker],
          ['Cost Per Action Taker', A_cost_per_action_taker],
          ['Total Agree to Relational Shares', A_share_agree],
          ['% of Total Agree to Relational Shares', A_percent_share_agree],
          ['Cost Per Agree to Relational share', A_cost_per_share_agree],
          ['Live Chat engagements', live_chat],
          
          
          
        ]
 
# Create the pandas DataFrame
A_GoToVacc_metrics = pd.DataFrame(A_new_df, columns = ['Description', 'Metric'])
A_GoToVacc_metrics

In [ ]:
# Group B report generation

B_new_df = [
          ['Total Ad Spend (USD)', B_fb_total_ad_spend],
          ['Total Subscribers Acquired FB (Results)', int(groupB_fb_results)],
          ['Total Subscribers Acquired CF', B_total_cf_subscribers],
          ['Cost Per Result FB Ads (USD)', B_fb_cost_per_result_alt_for],
          ['Cost Per Acquisition CF', B_cost_per_result_cf],
          ['Zip Codes Submitted', zip_codes_submitted],
          ['Cost Per Zip Code Submitted', B_cost_per_zip_code],
          ['% of Zip Codes submitted', B_percent_zip_codes_submitted],

          ['Total Vaccinated', B_vaccinated_cf],
          ['% of Vaccinated', B_percent_vaccinated],
          ['Total Not Vaccinated', B_not_vaccinated_cf],
          ['% of Not Vaccinated', B_percent_not_vaccinated],
          ['Vaccinated Action Takers', B_vaccinated_action_takers],
          ['% of Vaccinated Action Takerse (from Vaccinated Users)', B_percent_per_action_taker_vac],
          ['Not Vaccinated Action Takers', B_not_vac_action_takers],
          ['% of Not Vaccinated Action Takers (from Not Vaccinated Users', B_percent_per_action_taker_not_vac],
          ['Total Action Takers', B_total_action_takers],
          ['% of Action Takers', B_percent_per_action_taker],
          ['Cost Per Action Taker', B_cost_per_action_taker],
          ['Total Agree to Relational Shares', B_share_agree],
          ['% of Total Agree to Relational Shares', B_percent_share_agree],
          ['Cost Per Agree to Relational share', B_cost_per_share_agree],
          ['Live Chat engagements', live_chat],
          
          
          
        ]
 
# Create the pandas DataFrame
B_GoToVacc_metrics = pd.DataFrame(B_new_df, columns = ['Description', 'Metric'])
B_GoToVacc_metrics

In [ ]:
# EDA: vaccination status by county

cf_county = cf_data[['county_localyst', 'Vaccinated_Wisc_from_ad']]
breakdown_county_cf=cf_county.value_counts()
breakdown_county_cf

In [ ]:
# EDA: county by share agrees

cf_share = cf_data[['county_localyst', 'share_card_received']]
cf_share.value_counts()

**Here you need to click on the link and it will create a authentification token that you will plaste in the space provided in the code**

Click on the link that appears after "Go to this URL in a browser:"
Chose your google account, sign in, copy the token.
Paste the token in the slot provided in the script and click "enter".

In [ ]:
from datetime import datetime
from pytz import timezone
import pytz

def get_pst_time():
    date_format='%m_%d_%Y_%H_%M_%S_%Z'
    date = datetime.now(tz=pytz.utc)
    date = date.astimezone(timezone('US/Pacific'))
    pstDateTime=date.strftime(date_format)
    return pstDateTime


date_PDT=get_pst_time()
date_PDT

In [ ]:
today = date.today()
today = today.strftime("%b-%d-%Y")
today

In [ ]:
# Mount drive from google

from google.colab import drive
drive.mount('drive')

In [ ]:
# Export metrics report back to files
# The files will be in the same folder where you uploaded the data

GoToVacc_metrics.to_csv(f'{date_PDT}-Wisconsin GoToVacc Metrics.csv')
!cp WISC_GoToVacc_Metrics.csv "drive/My Drive/"

In [ ]:
A_GoToVacc_metrics.to_csv(f'{date_PDT}-A_Wisconsin GoToVacc Metrics.csv')
!cp A_WISC_GoToVacc_Metrics.csv "drive/My Drive/"

In [ ]:
B_GoToVacc_metrics.to_csv(f'{date_PDT}-B_Wisconsin GoToVacc Metrics.csv')
!cp B_WISC_GoToVacc_Metrics.csv "drive/My Drive/"

In [ ]:
# We will create a new dataframe with subscribers with zip codes

subscribers_zip_code = zip_code_list[['profile pic url','first name', 'last name','gender','zip_code', 'state_localyst', 'Vaccinated_Wisc_from_ad', 'wi_GTVac_action', 'county_localyst' ]]
subscribers_zip_code

In [ ]:
# We will create a new dataframe with subscribers with counties

county_data = A_cf_data.dropna(subset=['county_localyst'])
subscribers_county = county_data[['profile pic url','first name', 'last name','gender','county_localyst' , 'state_localyst', 'Vaccinated_Wisc_from_ad', 'wi_GTVac_action']]
subscribers_county

In [ ]:
# Export metrics report back to files
# The files will be in the same folder where you uploaded the data

subscribers_zip_code.to_csv(f'{date_PDT}-Zip_Codes_EP_Subscribers.csv')
!cp Zip_Codes_EP_Subscribers.csv "drive/My Drive/"

In [ ]:
# Export metrics report back to files
# The files will be in the same folder where you uploaded the data

subscribers_county.to_csv(f'{date_PDT}-County_EP_Subscribers.csv')
!cp County_EP_Subscribers.csv "drive/My Drive/"